In [8]:
import os 

In [9]:
%pwd

'c:\\Users\\Chaitanya\\Documents\\ML\\Indian Medical Leaf Clf\\Medicinal-Leaf-Classification\\research'

In [10]:
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\Chaitanya\\Documents\\ML\\Indian Medical Leaf Clf\\Medicinal-Leaf-Classification'

### entity (entity/config_entity.py)

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_model_path: Path
    params_image_size:list
    params_boundaries: list
    params_learning_rate:list
    params_include_top: bool
    params_weight:str
    params_classes: int
    params_freeze:bool

### Configuration (configuration.py)

In [13]:
from MedicineLeafClassifier.constants import *
from MedicineLeafClassifier.utils.common import read_yaml,create_directories

In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weight=self.params.WEIGHTS,
            params_boundaries=self.params.BOUNDARIES,   
            params_learning_rate=self.params.LR_VALUES,
            params_classes=self.params.CLASSES,
            params_freeze=self.params.FREEZE
        )

        return prepare_base_model_config
        

### Components (prepare_base_model.py)

In [15]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [16]:
class PrepareBasemodel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.applications.InceptionResNetV2(
            input_shape=self.config.params_image_size,
            weights = self.config.params_weight,
            include_top = self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)     


    @staticmethod
    def prepare_full_model(model,classes,freeze,boundaries,learning_rate):
        lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
            boundaries,learning_rate
        )
        
        if freeze:
            model.trainable = False

        full_model = tf.keras.Sequential([
            tf.keras.layers.Input((224,224,3)),
            tf.keras.layers.Rescaling(1./255),
            model, 
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(classes, activation='softmax')  # Adjust for your number of classes
            ])
        
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate = lr_schedule), 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy']
        )
        full_model.summary()

        return full_model
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze=self.config.params_freeze,
            boundaries=self.config.params_boundaries,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_model_path, model=self.full_model)

    @staticmethod
    def save_model(path:Path, model):
        model.save(path)

### Pipeline

In [17]:
try:
    config = ConfigurationManager()
    Prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBasemodel(config=Prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-02-29 13:21:21,102: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-29 13:21:21,107: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-29 13:21:21,107: INFO: common: Created directory at artifacts]
[2024-02-29 13:21:21,113: INFO: common: Created directory at artifacts/prepare_base_model]
[2024-02-29 13:21:27,312: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                             